In [1]:
from nansat import Nansat, Domain
import matplotlib.pyplot as plt
import numpy as np
from boreali import Boreali, lm
import custrepr
from matplotlib.patches import Rectangle

In [2]:
# different kinds of test data was processed by boreali algorithms 
# october data
n_osw_oct = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014301181500.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_oct = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014301181500.L2_LAC_OC.x.nccpa_deep.nc')
# may data
n_osw_may = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014121184000.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_may = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014121184000.L2_LAC_OC.x.nccpa_deep.nc')

# base files
n_rrs_oct = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014301181500.L2_LAC_OC.x.nc')
n_rrs_may = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014121184000.L2_LAC_OC.x.nc')

# bottom = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd')

dom_sbd = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 44.6 -85.2 45.3 -ts 300 200')
dom_reg = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 42.8 -86.2 42.6 -ts 150 300')

n_rrs_oct.reproject(dom_sbd)
n_rrs_may.reproject(dom_sbd)
bottom_sbd = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd')
bottom_reg = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd')

=>michigan.grd<=
=>michigan.grd<=


In [4]:
aaa = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd')
aaa

=>michigan.grd<=


----------------------------------------
/home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd
----------------------------------------
Mapper: generic
----------------------------------------
Band : 1 z
  _FillValue: -nan
  name: z
  dataType: 6
  actual_range: {-275.8900146484375,351.9434814453125}
  long_name: z
  SourceFilename: /home/artemm/Documents/Work/MichiganLake/TetsData/michigan_lld.grd
  SourceBand: 1
  wkv: 
----------------------------------------
Domain:[4201 x 5365]
----------------------------------------
Projection:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563],
        TOWGS84[0,0,0,0,0,0,0]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433]]
----------------------------------------
Corners (lon, lat):
	 (-88.00,  46.09)  (-84.50,  46.09)
	 (-88.00,  41.62)  (-84.50,  41.62)

In [86]:
# droving of bottom
#bottom_sbd.reproject(dom_sbd)
bottom_reg.reproject(dom_reg)


y_points = np.arange(0, 300, 150)
y_labels = np.arange(42.8, 42.6 -0.1)

x_points = np.arange(0, 150, 75)
x_labels = np.arange( 86.3, 86.2, -0.1)
#len(x_labels)


h = np.copy(bottom_reg[1])
h[np.where(h > np.float32(0.0))] = np.nan
h[np.where(h < np.float32(0.0))] *= np.float32(-1)
#h[np.where(h >= np.float32(10.0))] = np.nan
plt.figure('bottom')
#plt.text(50, 102, '1', fontsize=25, )
#plt.text(120, 30, '2', fontsize=25, )
#plt.text(222, 10, '3', fontsize=25, )

plt.imshow(h)
plt.yticks(y_points, y_labels)
plt.xticks(x_points, x_labels)
plt.grid(color='black')
plt.colorbar()

# adding of rectangles for each region
#currentAxis = plt.gca()
#currentAxis.add_patch(Rectangle((112 - .5, 0 - .5), 17, 20, fill=False))    # bank near S.Fox isl.
#currentAxis.add_patch(Rectangle((14 - .5, 86 - .5), 46, 48, fill=False))    # Maniton Passage
#currentAxis.add_patch(Rectangle((196 - .5, 14 - .5), 68, 38, fill=False))   # Grand Traverse Bay

plt.show()

/home/artemm/Programs/Miniconda/miniconda2/envs/niersc/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in less
/home/artemm/Programs/Miniconda/miniconda2/envs/niersc/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in greater_equal


In [54]:
# droving of Rrsw spectrum for the deep and shallow region 
num_bands = range(2, 12)
n = n_rrs_oct
deep_p_x, deep_p_y = 3, 26
shallow_p_x, shallow_p_y = 34, 119

test_deep = [n[band][deep_p_y][deep_p_x] for band in num_bands]
h_deep = bottom[1][deep_p_y][deep_p_x]
h_shallow = bottom[1][shallow_p_y][shallow_p_x]
test_shallow = [n[band][shallow_p_y][shallow_p_x] for band in num_bands]
wavelens = [412, 443, 469, 488, 531, 547, 555, 645, 667, 678]

custom_n = Nansat(domain=n)

# Rrsw processing
band_rrs_numbers = list(map(lambda x: n._get_band_number('Rrs_' + str(x)), wavelens))

for index in range(0, len(wavelens)):
    rrsw = n[band_rrs_numbers[index]] / (0.52 + 1.7 * n[band_rrs_numbers[index]])
    custom_n.add_band(rrsw, parameters={'name': 'Rrsw_' + str(wavelens[index]),
                                        'units': 'sr-1',
                                        'wavelength': wavelens[index]})

# plotting
plt.plot(num_bands, test_deep, label='deep ' + str(round(h_deep * -1, 1)) + 'm')
plt.plot(num_bands, test_shallow, label='shallow ' + str(round(h_shallow * -1, 1)) + 'm')
plt.legend()
plt.xticks(num_bands, wavelens)
plt.xlabel('wavelength, nm')
plt.ylabel('Rrs, sr^-1')
#plt.ylim([0, 0.04])
plt.show()

NameError: name 'bottom' is not defined

In [31]:
plt.imshow(n_osw_oct['chl'])
plt.colorbar()
plt.show()

In [6]:
h = np.copy(bottom[1])
h[np.where(h > np.float32(0.0))] = np.nan
h[np.where(h < np.float32(0.0))] *= np.float32(-1)

m_osw = []
m_deep = []
#for i in range(np.nanmin(h), np.nanmax(h)):


plt.imshow(n_osw_oct['chl'])
plt.colorbar()
plt.show()

/home/artemm/Programs/Miniconda/miniconda2/envs/niersc/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


In [6]:
dat = Nansat('/home/artemm/Documents/Work/MichiganLake/TetsData/A2014301181500.L2_LAC_OC.x.nc')
dom = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 44.6 -85.2 45.3 -ts 300 200')
dat.reproject(dom)

In [10]:
wavelen = [412, 443, 469, 488, 531, 547, 555, 645, 667, 678]
print wavelen

band_rrs_numbers = list(map(lambda x: dat._get_band_number('Rrs_' + str(x)), wavelen))
print band_rrs_numbers

custom_n = Nansat(domain=n)

for index in range(0, len(wavelen)):
    if 412 < wavelen[index] < 667:
        print wavelen[index]
        rrsw = n[band_rrs_numbers[index]] / (0.52 + 1.7 * n[band_rrs_numbers[index]])
        custom_n.add_band(rrsw, parameters={'name': 'Rrsw_' + str(wavelen[index]),
                                            'units': 'sr-1',
                                            'wavelength': wavelen[index]})
        
print custom_n

[412, 443, 469, 488, 531, 547, 555, 645, 667, 678]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
443
469
488
531


547
555
645
----------------------------------------

----------------------------------------
Mapper: 
----------------------------------------
Band : 1 Rrsw_443
  name: Rrsw_443
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 1
  wavelength: 443
Band : 2 Rrsw_469
  name: Rrsw_469
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 2
  wavelength: 469
Band : 3 Rrsw_488
  name: Rrsw_488
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 3
  wavelength: 488
Band : 4 Rrsw_531
  name: Rrsw_531
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 4
  wavelength: 531
Band : 5 Rrsw_547
  name: Rrsw_547
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 5
  wavelength: 547
Band : 6 Rrsw_555
  name: Rrsw_555
  dataType: 6
  units: sr-1
  SourceFilename: /vsimem/ITL2TAF9YW.vrt
  SourceBand: 6
  wavelength: 555
Band : 7 Rrsw_645
  name: R